In [13]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from secret import API_KEY
from data import DATA

In [14]:
def pieWinrate(df):
    plt.figure()
    plt.title("Winrate")
    percent = df['winrate'].mean()*100
    labels = ['win', 'lose']
    plt.pie([percent, 100-percent], labels=labels, autopct='%1.0f%%')
    plt.legend()
    plt.show()

In [15]:
def formatScore(nbr):
    level = ""
    if nbr < 0.2:
        level = "C'est pas vraiment pas ouf.."
    elif nbr < 0.4:
        level = "Ça vaaaaaaa"
    elif nbr < 0.6:
        level = "Il s'en sort bien !"
    elif nbr < 0.8:
        level = "Trop fort"
    else:
        level = "Il est juste gifted en faite"
    return f"T'es a {nbr:.0%} de performance. {level}"


In [16]:
def getPuuid(ig):
    return requests.get(f"https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{ig}?api_key={API_KEY}").json()["puuid"]

In [17]:
def getRecentGames(puuid):
    nb = 20
    return requests.get(f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count={nb}&api_key={API_KEY}").json()

In [18]:
def getDFfromJson(puuid, games, champ):
    array = []
    for i in games:
        json = requests.get(f"https://europe.api.riotgames.com/lol/match/v5/matches/{i}?api_key={API_KEY}").json()
        index = json['metadata']['participants'].index(puuid)
        player = json['info']['participants'][index]
        if player['championName'] != champ: continue
        if player['teamPosition'] != DATA[champ]["role"]: continue
        player['gameDuration'] = json['info']['gameDuration']
        array.append(pd.DataFrame({x: [player[x]] for x in DATA[champ]["columns"]}))
            
    return pd.concat(array).reset_index(drop=True)

In [19]:
def formatDf(df):
    df['scoreCreeps'] = df['totalMinionsKilled'] + df['neutralMinionsKilled']
    df.rename(columns={'win': 'winrate'})
    return df

In [20]:
def getParams(champ):
    def gen(n):
        for i in n:
            yield i

    generator = gen(open('./params.txt', 'r'))
    
    while True:
        str = next(generator)
        if champ in str:
            stats = str.strip().split(';')
            stats.pop(0)
            return {s.split(',')[0]: list(map(float, s.split(',')[1:])) for s in stats}

In [21]:
def calcScore(params, df):
    s = 0
    for k, v in params.items():
        n = (df[k] / (df['gameDuration'] / 60)).mean()
        s += v[0]*((n-v[1])/(v[2]-v[1]))
    return float(s)

In [22]:
def getScore(df, champ):
    params = getParams(champ)
    s = calcScore(params, df)
    return s

In [23]:
ig = "Vayntox"
champ = "Vayne"
puuid = getPuuid(ig)
games = getRecentGames(puuid)
df = getDFfromJson(puuid, games, champ)
df = formatDf(df)
params = getParams(champ)
score = getScore(df, champ)
print(formatScore(score))

T'es a 43.0% de performance. Tu t'en sors bien !
